In [11]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from alive_progress import alive_bar

In [16]:
data = pd.read_csv('data/random_address_merged.csv')
data.head(2)

,Address,Place,City,State,Phone,Zip,CountryCode,Country,Location
0,"['Street: 191-A, Pekan Jitra Ii,', 'City: J...","191-A, Pekan Jitra Ii,",Jitra,Kedah,+60 (0)3 2148-3833,6000,60,Malaysia,"191-A, Pekan Jitra Ii,, Jitra, Kedah"
1,['Street: 7Th Floor Sun Komplek Jln Bukit Bin...,7Th Floor Sun Komplek Jln Bukit Bintang,Kuala Lumpur,Wilayah Persekutuan,032144-9297,55100,60,Malaysia,"7Th Floor Sun Komplek Jln Bukit Bintang, Kuala..."


In [17]:
# testing geolocator library
geolocator = Nominatim(user_agent="u2004763@siswa.um.edu.my")
location = geolocator.geocode("Johor Bahru") # testing

print(location.point)
print(location.address)

# print out longitude and latitude
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

1 30m 12.798s N, 103 44m 58.411s E
Johor Bahru, Iskandar Malaysia, Johor, 80350, Malaysia
Latitude: 1.503555
Longitude: 103.7495586


In [18]:
# geocoding function
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return pd.Series({'Latitude': None, 'Longitude': None})

data[['Latitude', 'Longitude']] = data.apply(lambda x: my_geocoder(x['Location']), axis=1)

# check the percentage of data successfully geocoded
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

25.32727272727273% of addresses were geocoded!


In [19]:
# export file including NaN for checking
data.to_csv('data/random_address_geocoded_nan.csv', index=False)

In [20]:
# Drop location that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude))
data.crs = {'init': 'epsg:4326'}
data.head()

c:\Users\Khor Kean Teng\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Address,Place,City,State,Phone,Zip,CountryCode,Country,Location,Latitude,Longitude,geometry
6,"['Street: 1st Floor, Jalan Tun Perak,', 'City...","1st Floor, Jalan Tun Perak,",Kuala Lumpur,Wilayah Persekutuan,03-2026-6510,50050,60,Malaysia,"1st Floor, Jalan Tun Perak,, Kuala Lumpur, Wil...",3.149232,101.696630,POINT (101.69663 3.14923)
9,"['Street: A 116 Jln Serampang Taman Pelangi',...",A 116 Jln Serampang Taman Pelangi,Johor Bahru,Johor,07333-3296,80400,60,Malaysia,"A 116 Jln Serampang Taman Pelangi, Johor Bahru...",1.484403,103.779390,POINT (103.77939 1.48440)
16,"['Street: 69, Jalan Ali,', 'City: Muar', 'S...","69, Jalan Ali,",Muar,Johor,6069533808,84000,60,Malaysia,"69, Jalan Ali,, Muar, Johor",2.045842,102.567866,POINT (102.56787 2.04584)
21,"['Street: Jalan Kenari 17D, Bandar Puchong Ja...","Jalan Kenari 17D, Bandar Puchong Jaya,",Puchong,Selangor,+60 (0)3 8076-1325,47100,60,Malaysia,"Jalan Kenari 17D, Bandar Puchong Jaya,, Puchon...",3.047065,101.624011,POINT (101.62401 3.04707)
23,"['Street: Jalan Pjs 1/50, Taman Petaling Utam...","Jalan Pjs 1/50, Taman Petaling Utama,",Petaling Jaya,Selangor,+60 (0)3 7770-5556,46000,60,Malaysia,"Jalan Pjs 1/50, Taman Petaling Utama,, Petalin...",3.079325,101.647865,POINT (101.64787 3.07932)


In [22]:
# export csv for the successful geocoded data
data.to_csv('data/random_address_geocoded.csv', index=False)